##  AutoAIデータ作成用
データの内容も日本語化  
2020-09-23 訓練データ件数を減らす(Lite Plan対策)

In [ ]:
# 共通事前処理

# 必要ライブラリのimport
import pandas as pd
# データフレーム表示用関数
from IPython.display import display

In [ ]:
# 公開データのダウンロードと解凍
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip -O bank.zip | tail -n 1
!unzip -o bank.zip | tail -n 1

# bank-full.csvをデータフレームに取り込み
df_all = pd.read_csv('bank-full.csv', sep=';')

# 項目名を日本語に置き換える
columns = [
    '年齢', '職業', '婚姻', '学歴', '債務不履行',  '平均残高',  '住宅ローン',  '個人ローン',
    '連絡手段',  '最終通話日',  '最終通話月',  '最終通話秒数', '通話回数_販促中',
    '前回販促後_経過日数',  '通話回数_販促前',  '前回販促結果', '今回販促結果'
]
df_all.columns = columns

In [ ]:
display(df_all.head())

In [ ]:
yn_dict = {
    'yes': 'はい',
    'no': 'いいえ'
}
yn_dict2 = {
    'yes': '成功',
    'no': '失敗'
}    

In [ ]:
df_jp = df_all.copy()
df_jp['債務不履行'] = df_jp['債務不履行'].map(yn_dict)
df_jp['住宅ローン'] = df_jp['住宅ローン'].map(yn_dict)
df_jp['個人ローン'] = df_jp['個人ローン'].map(yn_dict)
df_jp['今回販促結果'] = df_jp['今回販促結果'].map(yn_dict2)



In [ ]:
display(df_jp.head())

In [ ]:
random_seed = 123
# 訓練データと検証データの分割
# 訓練データ10% 検証データ90%の比率で分割する
test_size = 0.90

y = df_jp['今回販促結果'].values

from sklearn.model_selection import train_test_split
df_jp_train, df_jp_test = train_test_split(
  df_jp, test_size=test_size, random_state=random_seed,
  stratify=y)

In [ ]:
print(df_jp_train.shape)
print(df_jp_test.shape)

### CSVファイル保存

In [ ]:
fn_train = 'bank-train-jp-autoai.csv'
df_jp_train.to_csv(fn_train, index=False)

In [ ]:
fn_test = 'bank-test-jp-autoai.csv'
df_jp_test.to_csv(fn_test, index=False)

### Google Colabからダウンロード

In [ ]:
try:
    from google.colab import files
    files.download(fn_train) 
    files.download(fn_test)      
except:
    print('not google colab')

### 以下はjsonデータ作成用

In [ ]:
df_jp_yes = df_jp_test[df_jp_test['今回販促結果']=='成功']
display(df_jp_yes.head())

In [ ]:
df_jp_no = df_jp_test[df_jp_test['今回販促結果']=='失敗']
display(df_jp_no.head())

In [ ]:
df_jp_yes20 = df_jp_yes.drop('今回販促結果', axis=1).head(20)
df_jp_no20 = df_jp_no.drop('今回販促結果', axis=1).head(20)

In [ ]:
fields = df_jp_yes20.columns.tolist()
print(fields)

In [ ]:
values_yes = df_jp_yes20.values.tolist()
print(values_yes)

In [ ]:
values_no = df_jp_no20.values.tolist()
print(values_no)

In [ ]:
import json

In [ ]:
scoring_payload_yes = {"input_data": [{
   "fields": fields, 
    "values": values_yes
}]}

print(json.dumps(scoring_payload_yes,  ensure_ascii=False))

In [ ]:
scoring_payload_no = {"input_data": [{
   "fields": fields, 
    "values": values_no
}]}

print(json.dumps(scoring_payload_no,  ensure_ascii=False))

In [ ]:
print(df_jp_test.shape)